In [1]:
import time
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    return_messages=True
)

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# loader = UnstructuredFileLoader("./files/chapter_one.txt")
loader = UnstructuredFileLoader("../files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("../.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. 
            If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]

    # case1
    # return "\n\n".join(
    #     map_doc_chain.invoke(
    #         {"context": doc.page_content, "question": question}
    #     ).content
    #     for doc in documents
    # )
    # case2
    results = []
    for document in documents:
        result = map_doc_chain.invoke(
            {"context": document.page_content, "question": question}
        ).content
        results.append(result)
        # for RateLimit issue
        time.sleep(20)
    return "\n\n".join(results)


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            Please answer everything in Korean.
            ------
            {context}
            """,
        ),
        (
            "system",
            "The above examples should not be provided to the user. The user can only be provided with the conversation record below. Please provide the information to the user using the record below.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"context": map_chain, "question": RunnablePassthrough(), "history": load_memory } | final_prompt | llm

In [2]:
chain.invoke("Is Aaronson guilty?")
# chain.invoke("Aaronson 은 유죄인가요?")

AIMessage(content='예, Aaronson은 그의 범행에 유죄입니다.')

In [3]:
chain.invoke("What message did he write in the table?")
# chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")

AIMessage(content='그는 슬레이트 테이블에 "자유는 노예다"라고 썼고, 그 다음에 "두 더하기 두는 다섯이다"라고 썼습니다.')

In [4]:
chain.invoke("Who is Julia?")
# chain.invoke("Julia 는 누구인가요?")

AIMessage(content='주리아는 주인공이 사랑을 고백하는 장면에서 언급되는 캐릭터입니다. 주인공은 자신을 구하기 위해 주리아에게 벌을 받도록 소리쳤다고 합니다. 주리아는 주인공과 로맨틱하거나 개인적인 연결이 있음을 시사하는 인물로 언급되었습니다.')